## Get Best-Performing Hyper Parameters

In [1]:
MONGO_COLLECTION = "SC_RE-RANKER_HYPER_PARAM_VD"

In [2]:
# for r in rows[-17:]:
#     print(str(r["asof"]))

In [3]:
cols = ["f1_score", "precision", "recall", "asof", "best_top_n", "C", "best_max_parses", 
    #"best_max_upd", 
    "max_update_items", 
    # "best_min_prob", "extractors", 
    "initial_weight", "loss_type",\
    #"min_feat_freq",
    "pa_type"]

In [4]:
import pandas as pd
import pymongo

client = pymongo.MongoClient(serverSelectionTimeoutMS=100, host="127.0.0.1")
db = client.metrics_causal_model_reranker

project = {
        "params": "$parameters",
        "micro_f1": "$MICRO_F1",
        "asof": "$asof",
        "_id": 1
    }
feats_pipeline = [{ "$project": project }]
rows = [row for row in db[MONGO_COLLECTION].aggregate(feats_pipeline)]

results = []
for r in rows:
    d = dict(r["params"])
    d.update(r["micro_f1"])
    d["asof"] = str(r["asof"])
    results.append(d)

In [5]:
df = pd.DataFrame(results)
df.tail(3)[["asof"] + cols]

,asof,f1_score,precision,recall,asof,best_top_n,C,best_max_parses,max_update_items,initial_weight,loss_type,pa_type
0,2019-06-18 07:50:02.014000,0.805714,0.869093,0.750951,2019-06-18 07:50:02.014000,1,0.0005,300,1,0.01,pb,1


In [6]:
df = pd.DataFrame(results)
df = df.sort_values(by="f1_score",ascending=False)
df[cols].head(5)

,f1_score,precision,recall,asof,best_top_n,C,best_max_parses,max_update_items,initial_weight,loss_type,pa_type
0,0.805714,0.869093,0.750951,2019-06-18 07:50:02.014000,1,0.0005,300,1,0.01,pb,1


In [7]:
# Best ones that are not 2
# df[df.best_top_n != 2][cols].head()